In [57]:
# tool create

from dotenv import load_dotenv
load_dotenv()

True

In [58]:
from langchain_core.tools import tool
from langchain_core.tools import InjectedToolArg
from typing import Annotated
import requests
@tool
def get_conversion_factor(base_currency:str, target_currency: str) -> float:
    """
    This func fetches  the conversion factor between the base and target currency
    """
    url = f"https://v6.exchangerate-api.com/v6/02875630216864e58293a136/pair/{base_currency}/{target_currency}"

    response = requests.get(url)
    return response.json()

@tool 
def convert(base_currency_value:int, conversion_rate: Annotated[float,InjectedToolArg]) -> float:
    """
    given a currency conversion rate this func calculates the target currency value from given base currency value
    """

    return base_currency_value*conversion_rate

In [59]:
get_conversion_factor.invoke({'base_currency':"USD", "target_currency":"INR"})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1760572801,
 'time_last_update_utc': 'Thu, 16 Oct 2025 00:00:01 +0000',
 'time_next_update_unix': 1760659201,
 'time_next_update_utc': 'Fri, 17 Oct 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 88.0112}

In [60]:
convert.invoke({'base_currency_value':10, "conversion_rate":88.0112})

880.1120000000001

In [61]:
# tool binding
from langchain_google_genai import ChatGoogleGenerativeAI
import os
llm = ChatGoogleGenerativeAI(
    model = 'models/gemini-2.5-pro'
)


In [62]:
llm_with_tools= llm.bind_tools([get_conversion_factor, convert])

In [63]:
from langchain_core.messages import HumanMessage
messages = [HumanMessage("convert 10 USD to INR")]
messages

[HumanMessage(content='convert 10 USD to INR', additional_kwargs={}, response_metadata={})]

In [64]:
ai_message = llm_with_tools.invoke(messages)
ai_message.tool_calls[0]

{'name': 'get_conversion_factor',
 'args': {'target_currency': 'INR', 'base_currency': 'USD'},
 'id': 'b579d038-19ad-4b2c-a334-4f63244597fc',
 'type': 'tool_call'}

In [66]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage("convert 10 USD to INR")]
ai_message = llm_with_tools.invoke(messages)

print(ai_message)          # full object
print(getattr(ai_message, "tool_calls", None))  # explicitly see if tool_calls exist


content='I can do that. What is the conversion rate from USD to INR?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-pro', 'safety_ratings': []} id='run--0ae1d6b8-f369-431a-a199-bfaa7af96880-0' usage_metadata={'input_tokens': 129, 'output_tokens': 767, 'total_tokens': 896, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 752}}
[]


In [72]:
# from langchain_core.messages import ToolMessage

# # model_request
# messages = [HumanMessage("""
#  Use the tools provided.
#         First, call get_conversion_factor to get the rate between USD and INR. 
#         Then, call convert to calculate 10 USD in INR.
# """)]

# # model decides which tool to call
# ai_message = llm_with_tools.invoke(messages)
# print(ai_message)


# # Run the tool
# tool_call = ai_message.tool_calls[0]

# if tool_call['name'] == 'get_conversion_factor':
#     result = get_conversion_factor.invoke(tool_call["args"])
    
#     messages.append(ai_message)
#     messages.append(ToolMessage(content=str(result), tool_call_id=tool_call["id"]))


#     follow_up = llm_with_tools.invoke(messages)
#     print("second Tool calls", follow_up.tool_calls)

# # EXECUTE 
# second_call = follow_up.tool_calls[0]
# if second_call["name"] == "convert":
#     result2= convert.invoke(second_call["args"])

#     messages.append(follow_up)
#     messages.append(ToolMessage(content=str(result2), tool_call_id = second_call["id"]))

#     #final response

#     final_response = llm_with_tools.invoke(messages)
#     print("Final response: ", final_response.content)

from langchain_core.messages import HumanMessage, ToolMessage

# Step 1 — Initial request
messages = [HumanMessage("""
Use the tools provided.
First, call get_conversion_factor to get the rate between USD and INR.
Then, call convert to calculate 10 USD in INR.
""")]

# Step 2 — Model chooses first tool
ai_message = llm_with_tools.invoke(messages)
print("First tool calls:", ai_message.tool_calls)

# Step 3 — Execute get_conversion_factor
tool_call = ai_message.tool_calls[0]
result = get_conversion_factor.invoke(tool_call["args"])

# Extract the conversion rate
rate = result["conversion_rate"]

# Feed result back
messages.append(ai_message)
messages.append(ToolMessage(content=str(result), tool_call_id=tool_call["id"]))

# Step 4 — Model follows up (may call convert)
follow_up = llm_with_tools.invoke(messages)
print("Second tool calls:", follow_up.tool_calls)

# Step 5 — Execute convert (manually fix missing rate)
second_call = follow_up.tool_calls[0]

# Inject the conversion_rate if missing
if "conversion_rate" not in second_call["args"]:
    second_call["args"]["conversion_rate"] = rate

# Execute convert
result2 = convert.invoke(second_call["args"])

# Feed that result back
messages.append(follow_up)
messages.append(ToolMessage(content=str(result2), tool_call_id=second_call["id"]))

# Step 6 — Final model output
final_response = llm_with_tools.invoke(messages)
print("💬 Final response:", final_response.content)


First tool calls: [{'name': 'get_conversion_factor', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': 'd13d619f-4662-478e-98cf-6436fe47fb36', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency_value': 10}, 'id': '5a2bab28-e77e-433e-80ad-7f8a9a421235', 'type': 'tool_call'}]
Second tool calls: [{'name': 'convert', 'args': {'base_currency_value': 10, 'conversion_rate': 88.0112}, 'id': '7af1c164-108a-4fa1-b242-a3a9f6f0178d', 'type': 'tool_call'}]
💬 Final response: 10 USD is equal to 880.112 INR.
